# Mapping strava routes

Point the notebook to the folder where the gpx are sitting

In [ ]:
my_folder = '/Users/ushhamilton/Documents/40_Map_Data/Cycle Touring/2022/gpx/'

In [ ]:
import os
import math
import pandas as pd
import numpy as np
import gpxpy as gp
from gpxpy.geo import distance
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
data_files = [] # populate this list if you want to specify only certain files in the folder

if len(data_files) == 0:
    data_files = os.listdir(my_folder)

data_files

In [ ]:
gpx_files = list()
for file in data_files:
    gpx_file = open(my_folder + '/' + file, 'r')
    gpx_files.append(gp.parse(gpx_file))

In [ ]:
temp_data = list()
for gpx in gpx_files:
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                temp_data.append([point.time, point.latitude, point.longitude, point.elevation])

In [ ]:
total_data = pd.DataFrame(temp_data, columns=['time', 'lat', 'lon', 'ele'])
total_data['date'] = total_data['time'].dt.date
total_data = total_data.sort_values(by=['time'])

# Calculate distance
temp_dist = [0]
temp_ele = [0]
for (ix1, row1), (ix2, row2) in zip(total_data.iloc[:-1].iterrows(), total_data.iloc[1:].iterrows()):
    temp_dist.append(distance(
                        latitude_1=row1['lat'],
                        longitude_1=row1['lon'], 
                        elevation_1=None,
                        latitude_2=row2['lat'],
                        longitude_2=row2['lon'], 
                        elevation_2=None))
    temp_ele.append(row2['ele'] - row1['ele'])

total_data['dist'] = temp_dist
total_data['ele_delta'] = temp_ele
total_data['CumDist'] = total_data['dist'].cumsum()

total_data.head()

We need to filter these points as there are too many to plot, and we only need a point every 100 or so metres

In [ ]:
def filter_points(df, gaps=100):
    # Defualt to taking a point every 100 metres
    total_data['CumDist_div'] = (total_data['CumDist'] / gaps).astype(int)
    filtered = total_data[total_data['CumDist_div'] != total_data['CumDist_div'].shift(1)]
    return filtered.drop('CumDist_div', axis=1)

filtered_data = filter_points(total_data)

In [ ]:
filtered_data

In [ ]:
cmap = mpl.colormaps['Reds']
filtered_data['perc_ele'] = filtered_data['ele'] / filtered_data['ele'].max()


In [ ]:
lat, lon, ele = filtered_data['lat'].to_list(), filtered_data['lon'].to_list(), filtered_data['ele'].to_list()
segs = np.array([[(lon[i], lat[i]), (lon[i+1], lat[i+1])] for i in range(len(lat)-1)])

In [ ]:
def roundup(x, nearest):
    return int(math.ceil(x / nearest)) * nearest

cols = mpl.collections.LineCollection(segs, cmap=cmap, norm=mpl.colors.Normalize(0, roundup(np.max(ele), 100)), transform=ccrs.PlateCarree(), linewidth=4)
cols.set_array(ele[:-1])

fig = plt.figure(figsize=(12, 8))

boarder = 1
map_extent = [filtered_data['lon'].min() - boarder, filtered_data['lon'].max() + boarder, filtered_data['lat'].min() - boarder, filtered_data['lat'].max() + boarder]

ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent(map_extent, crs=ccrs.PlateCarree())
# ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)


ax.add_collection(cols)
plt.colorbar(cols, ax=ax, ticks=[0, roundup(np.max(ele)/2, 100), roundup(np.max(ele), 100)])


plt.show()